# Developing LLM Applications with LangChain - Part 2

## Chains and Agents

Time to level up your LangChain chains! You'll learn to use the LangChain Expression Language (LCEL) for defining chains with greater flexibility. You'll create sequential chains, where inputs are passed between components to create more advanced applications. You'll also begin to integrate agents, which use LLMs for decision-making.

In [ ]:
import os

# Set your OpenAI API key
openai_api_key = os.environ['OPENAI_API_KEY']

### Building prompts for sequential chains
Over the next couple of exercises, you'll work to create a system for helping people learn new skills. This system needs to be built sequentially, so learners can modify plans based on their preferences and constraints. You'll utilize your LangChain LCEL skills to build a sequential chain to build this system, and the first step is to design the prompt templates that will be used by this system.

In [2]:
from langchain_core.prompts import PromptTemplate

# Create a prompt template that takes an input activity
learning_prompt = PromptTemplate(
    input_variables=["activity"],
    template="I want to learn how to {activity}. Can you suggest how I can learn this step-by-step?"
)

# Create a prompt template that places a time constraint on the output
time_prompt = PromptTemplate(
    input_variables=["learning_plan"],
    template="I only have one week. Can you create a plan to help me hit this goal: {learning_plan}."
)

# Invoke the learning_prompt with an activity
print(learning_prompt.invoke(
    {
        "activity": "play golf"
    }
))

text='I want to learn how to play golf. Can you suggest how I can learn this step-by-step?'


### Sequential chains with LCEL
With your prompt templates created, it's time to tie everything together, including the LLM, using chains and LCEL. An llm has already been defined for you that uses OpenAI's gpt-4o-mini model

For the final step of calling the chain, feel free to insert any activity you wish! If you're struggling for ideas, try inputting "play the harmonica".

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers.string import StrOutputParser

learning_prompt = PromptTemplate(
    input_variables=["activity"],
    template="I want to learn how to {activity}. Can you suggest how I can learn this step-by-step?"
)

time_prompt = PromptTemplate(
    input_variables=["learning_plan"],
    template="I only have one week. Can you create a plan to help me hit this goal: {learning_plan}."
)

llm = ChatOpenAI(model="gpt-4o-mini", api_key=openai_api_key)

# Complete the sequential chain with LCEL
seq_chain = (
    {
        "learning_plan": learning_prompt 
        | llm 
        | StrOutputParser()
    }
    | time_prompt
    | llm
    | StrOutputParser()
)

# Call the chain
print(seq_chain.invoke({"activity": "play the harmonica"}))

Here's a focused one-week plan to help you get started on your harmonica journey, broken down day by day. Each day will build upon the last to maximize your learning.

### One-Week Harmonica Learning Plan

#### **Day 1: Choose and Familiarize**
- **Choose Your Harmonica**: Ensure you have a diatonic harmonica in the key of C.
- **Familiarize Yourself**:
  - Learn the parts of the harmonica (reed plates, cover plates, comb).
  - Practice holding the harmonica correctly.
  - Focus on breath control: practice inhaling and exhaling through the harmonica.

#### **Day 2: Basic Techniques**
- **Single Notes**: 
  - Practice isolating single notes using tongue blocking.
  - Experiment with lip pursing to enhance sound clarity.
- **Blow and Draw Notes**: 
  - Learn the difference between blowing and drawing notes. Start with holes 1-5.

#### **Day 3: Simple Melodies**
- **Play Simple Songs**: 
  - Work on "Mary Had a Little Lamb" or "Twinkle Twinkle Little Star." 
  - Focus on accuracy and timi

### ReAct agents
Time to have a go at creating your own ReAct agent! Recall that ReAct stands for Reason and Act, which describes how they make decisions. In this exercise, you'll load the built-in wikipedia tool to integrate external data from Wikipedia with your LLM. An llm has already been defined for you that uses OpenAI's gpt-4o-mini model

Note: The wikipedia tool requires the wikipedia Python library to be installed as a dependency, which has been done for you in this case.

In [13]:
from langchain_openai import ChatOpenAI
from langchain.agents import load_tools
from langgraph.prebuilt import create_react_agent

llm = ChatOpenAI(model="gpt-4o-mini", api_key=openai_api_key)

# Define the tools
tools = load_tools(["wikipedia"])

# Examine tool
print('Tool name:', tools[0].name)
print('Tool description:', tools[0].description)
print('Tool return direct:', tools[0].return_direct)
print('Tool args:', tools[0].args)

# Define the agent
agent = create_react_agent(llm, tools)

# Invoke the agent
response = agent.invoke({"messages": [("human", "How many people live in New York City?")]})
print(response)
print(response['messages'][-1].content)

Tool name: wikipedia
Tool description: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
Tool return direct: False
Tool args: {'query': {'description': 'query to look up on wikipedia', 'title': 'Query', 'type': 'string'}}
{'messages': [HumanMessage(content='How many people live in New York City?', additional_kwargs={}, response_metadata={}, id='29cab2e6-0add-46a9-986f-e21e9b1adb45'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_355TpIhs2T9Hizp7LaPsKfA8', 'function': {'arguments': '{"query":"Demographics of New York City"}', 'name': 'wikipedia'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 90, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'p

### Defining a function for tool use
You're working for a SaaS (software as a service) company with big goals for rolling out tools to help employees at all levels of the organization to make data-informed decisions. You're creating a PoC for an application that allows customer success managers to interface with company data using natural language to retrieve important customer data.

You've been provided with a pandas DataFrame called customers that contains a small sample of customer data. Your first step in this project is to define a Python function to extract information from this table given a customer's name. pandas has already been imported as pd.

In [20]:
import pandas as pd
import io


customers_str = '''
id,name,subscription_type,active_users,auto_renewal
101,Tech Innovators Inc.,Premium,450,True
102,Green Solutions Ltd.,Standard,300,False
103,Global Enterprises,Basic,150,True
104,Peak Performance Co.,Premium,800,True
105,Visionary Ventures,Standard,600,False
106,NextGen Technologies,Basic,200,True
107,Dynamic Dynamics LLC,Premium,700,True
108,Infinity Services,Standard,500,False
109,Eco-Friendly Products,Basic,100,True
110,Future Insights,Premium,900,True
'''

customers = pd.read_csv(io.StringIO(customers_str))

# Define a function to retrieve customer info by-name
def retrieve_customer_info(name: str) -> str:
    """Retrieve customer information based on their name."""
    # Filter customers for the customer's name
    customer_info = customers[customers['name'] == name]
    return customer_info.to_string()
  
# Call the function on Peak Performance Co.
print(retrieve_customer_info("Peak Performance Co."))

    id                  name subscription_type  active_users  auto_renewal
3  104  Peak Performance Co.           Premium           800          True


### Creating custom tools
Now that you have a function for extracting customer data from the customers DataFrame, it's time to convert this function into a tool that's compatible with LangChain agents.

In [21]:
from langchain_core.tools import tool

# Convert the retrieve_customer_info function into a tool
@tool
def retrieve_customer_info(name: str) -> str:
    """Retrieve customer information based on their name."""
    customer_info = customers[customers['name'] == name]
    return customer_info.to_string()
  
# Print the tool's arguments
print(retrieve_customer_info.args)

{'name': {'title': 'Name', 'type': 'string'}}


### Integrating custom tools with agents
Now that you have your tools at-hand, it's time to set up your agentic workflow! You'll again be using a ReAct agent, which, recall, reasons on the steps it should take, and selects tools using this context and the tool descriptions. An llm has already been defined for you that uses OpenAI's gpt-4o-mini model

In [22]:
from langgraph.prebuilt import create_react_agent

# Create a ReAct agent
agent = create_react_agent(llm, [retrieve_customer_info])

# Invoke the agent on the input
messages = agent.invoke({"messages": [("human", "Create a summary of our customer: Peak Performance Co.")]})
print(messages['messages'][-1].content)

**Customer Summary: Peak Performance Co.**

- **ID:** 104
- **Name:** Peak Performance Co.
- **Subscription Type:** Premium
- **Active Users:** 800
- **Auto Renewal:** Yes
